In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE

In [2]:
from zipfile import ZipFile

zip_file = ZipFile('cc_fraud.zip')
df = pd.read_csv(zip_file.open('fraudTrain.csv'))
df.head(1)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0


In [3]:
df.shape

(1296675, 23)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [5]:
df.trans_date_trans_time.iloc[1]

'2019-01-01 00:00:44'

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [7]:
categoricals = ['merchant','category','first','last','gender','street','city','state','job']

In [8]:
# df.category.value_counts()

# Decide features to drop. 
for sure: ['Unnamed: 0','cc_num','first','last','street','city','state','trans_num','trans_date_trans_time']

In [9]:
df = df.drop(columns=['Unnamed: 0','cc_num','first','last','street','city','state','trans_num','trans_date_trans_time'])
df.head()

,merchant,category,amt,gender,zip,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,is_fraud
0,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,1325376018,36.011293,-82.048315,0
1,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1325376044,49.159047,-118.186462,0
2,fraud_Lind-Buckridge,entertainment,220.11,M,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,1325376051,43.150704,-112.154481,0
3,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,1325376076,47.034331,-112.561071,0
4,fraud_Keeling-Crist,misc_pos,41.96,M,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,1325376186,38.674999,-78.632459,0


In [11]:
def get_target_and_features(df):
    target = df.columns.to_list()[-1:]
    features = df.columns.to_list()[:-1]
    return target,features

In [12]:
target,features = get_target_and_features(df)
target

['is_fraud']

In [13]:
def train_val(df):
    y = df[target]
    x = df[features]
    x_train, x_validate, y_train, y_validate = train_test_split(x,y,test_size=.30, random_state=42)
    return x_train,x_validate,y_train,y_validate

In [14]:
x_train,x_validate,y_train,y_validate = train_val(df)

In [15]:
x_train.sample()

,merchant,category,amt,gender,zip,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long
878232,"fraud_Turcotte, Batz and Buckridge",health_fitness,158.61,F,84325,41.71,-111.9817,2078,"Scientist, audiological",1963-06-13,1356133741,42.246578,-111.049833


# Need to encode here. (pd.get_dummies)

In [16]:
dummy_train
dummy_val
# dummy_test - need to create test df since this was previously split and exists as its own dataset

NameError: name 'dummy_train' is not defined

In [ ]:
xtrain_smote = df[['lat','long','amt','city_pop','merch_lat','merch_long']]
xvalidate_smote = df[['lat','long','amt','city_pop','merch_lat','merch_long']]
xtrain_smote.sample()

In [ ]:
y = df[target]
x = df[features]
x_train, x_validate, y_train, y_validate = train_val(df)

In [ ]:
x_train.head(1)

In [ ]:
sns.histplot(y_train)
plt.title("Class Imbalance: is_Fraud")

In [ ]:
y_train.value_counts()

In [ ]:
from imblearn.over_sampling import SMOTENC

sm = SMOTENC(random_state = 42,categorical_features = categoricals)
xtrain_resample, ytrain_resample = sm.fit_resample(x_train, y_train)
xval_resample, yval_resample = sm.fit_resample(x_validate,y_validate)

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 42)
xtrain_resample, ytrain_resample = sm.fit_resample(x_train, y_train)
xval_resample, yval_resample = sm.fit_resample(x_validate,y_validate)

In [ ]:
plot_resample(x_train,y_train,xtrain_resample,ytrain_resample,"SMOTE REBALANCING")